In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Diversity Dataset

In [2]:
diversity_csv = "Resources/diversityindex.csv"
diversity_df = pd.read_csv(diversity_csv)
diversity_df

,Location,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013"
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9
...,...,...,...,...,...,...,...,...,...
3190,"Osage County, MO",0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1
3191,"Lincoln County, WV",0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2
3192,"Leslie County, KY",0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2
3193,"Blaine County, NE",0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8


In [48]:
# Create a a new df with info we need 

# Split the County coloum into State and County
new_df = diversity_df
new_df['County'], new_df['State'] = diversity_df['Location'].str.split(',').str
new_df.tail()

# Drops all rows that have NaN as County as States got split into County above. 
new_df = new_df.dropna()

#checking if this worked and drop was sucessful in removing STATe names from county coloum 

# dsd = df[df['County'] == 'TEXAS']
# dsd.tail()

new_df.head()

C:\Users\jagno\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """


,Location,Diversity-Index,"Black or African American alone, percent, 2013","American Indian and Alaska Native alone, percent, 2013","Asian alone, percent, 2013","Native Hawaiian and Other Pacific Islander alone, percent,","Two or More Races, percent, 2013","Hispanic or Latino, percent, 2013","White alone, not Hispanic or Latino, percent, 2013",County,State
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2,Aleutians West Census Area,AK
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7,Queens County,NY
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5,Maui County,HI
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2,Alameda County,CA
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9,Aleutians East Borough,AK


In [49]:

new_df2 = pd.DataFrame({"Diversity-Index":new_df.groupby(["State","County"])["Diversity-Index"].mean(),
                       "Black":new_df.groupby(["State","County"])["Black or African American alone, percent, 2013"].mean(),
                       "American Indian":new_df.groupby(["State","County"])["American Indian and Alaska Native alone, percent, 2013"].mean(),
                       "Asian":new_df.groupby(["State","County"])["Asian alone, percent, 2013"].mean(),
                       "Native Hawaiian":new_df.groupby(["State","County"])["Native Hawaiian and Other Pacific Islander alone, percent,"].mean(),
                       "Two or More Races":new_df.groupby(["State","County"])["Two or More Races, percent, 2013"].mean(),
                       "Hispanic/ Latino":new_df.groupby(["State","County"])["Hispanic or Latino, percent, 2013"].mean(),
                       "White":new_df.groupby(["State","County"])["White alone, not Hispanic or Latino, percent, 2013"].mean()})

In [23]:
new_df2.head()

Diversity-Index  Black  American Indian  \
State County                                                                
 AK   Aleutians East Borough             0.738867    7.7             21.8   
      Aleutians West Census Area         0.769346    7.4             13.8   
      Anchorage Municipality             0.601515    6.3              8.1   
      Bethel Census Area                 0.315556    0.7             81.8   
      Bristol Bay Borough                0.648271    0.0             33.8   

                                  Asian  Native Hawaiian  Two or More Races  \
State County                                                                  
 AK   Aleutians East Borough       41.4              0.7                3.7   
      Aleutians West Census Area   31.1              2.3                4.8   
      Anchorage Municipality        8.9              2.3                7.8   
      Bethel Census Area            1.0              0.3                4.1   
      Bristol Bay Borough           1.1              0.3               17.5   

                                  Hispanic/ Latino  White  
State County                                               
 AK   Aleutians East Borough                  13.5   12.9  
      Aleutians West Census Area              14.6   29.2  
      Anchorage Municipality                   8.6   60.5  
      Bethel Census Area                       1.6   11.5  
      Bristol Bay Borough                      3.9   45.3

# Unemployment Dataset

In [24]:
unemp_csv = "Resources/unemployment.csv"
unemp_df = pd.read_csv(unemp_csv)

unemp_df.head()

,Year,Month,State,County,Rate
0,2015,February,Mississippi,Newton County,6.1
1,2015,February,Mississippi,Panola County,9.4
2,2015,February,Mississippi,Monroe County,7.9
3,2015,February,Mississippi,Hinds County,6.1
4,2015,February,Mississippi,Kemper County,10.6


In [25]:
unemp_df = unemp_df[['County', 'State', 'Rate']]
unemp_df.head(10)

,County,State,Rate
0,Newton County,Mississippi,6.1
1,Panola County,Mississippi,9.4
2,Monroe County,Mississippi,7.9
3,Hinds County,Mississippi,6.1
4,Kemper County,Mississippi,10.6
5,Calhoun County,Mississippi,6.9
6,Clarke County,Mississippi,7.9
7,Jefferson County,Mississippi,14.3
8,Madison County,Mississippi,4.5
9,Sharkey County,Mississippi,11.1


In [26]:
us_state_abbrev = {
    
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [27]:
unemp_df['State'] = unemp_df['State'].map(us_state_abbrev).fillna(unemp_df['State'])
unemp_df.head(10)

,County,State,Rate
0,Newton County,MS,6.1
1,Panola County,MS,9.4
2,Monroe County,MS,7.9
3,Hinds County,MS,6.1
4,Kemper County,MS,10.6
5,Calhoun County,MS,6.9
6,Clarke County,MS,7.9
7,Jefferson County,MS,14.3
8,Madison County,MS,4.5
9,Sharkey County,MS,11.1


In [28]:
unemp_df_new = pd.DataFrame({"Unemployment Rate":unemp_df.groupby(["State","County"])["Rate"].mean()})
unemp_df_new

Unemployment Rate
State County                              
AL    Autauga County              5.417284
      Baldwin County              5.364198
      Barbour County              8.460185
      Bibb County                 7.684877
      Blount County               5.164506
...                                    ...
WY    Sweetwater County           4.749691
      Teton County                3.645062
      Uinta County                5.538580
      Washakie County             4.995062
      Weston County               4.448765

[2885 rows x 1 columns]

# Median Income Dataset

In [29]:
median_csv = "Resources/medianincome.csv"
median_df = pd.read_csv(median_csv)
median_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   County-State             3220 non-null   object 
 1   County                   3220 non-null   object 
 2   Population               3220 non-null   int64  
 3   Median household income  3219 non-null   float64
 4   State Code               3220 non-null   object 
 5   State                    3220 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 151.1+ KB


In [30]:
median_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   County-State             3220 non-null   object 
 1   County                   3220 non-null   object 
 2   Population               3220 non-null   int64  
 3   Median household income  3219 non-null   float64
 4   State Code               3220 non-null   object 
 5   State                    3220 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 151.1+ KB


In [31]:
median_df = median_df[['County',"State Code", 'Population',"Median household income"]]
median_df.head()

,County,State Code,Population,Median household income
0,Autauga County,AL,55221,51281.0
1,Baldwin County,AL,195121,50254.0
2,Barbour County,AL,26932,32964.0
3,Bibb County,AL,22604,38678.0
4,Blount County,AL,57710,45813.0


In [32]:
median_df = median_df.rename(columns={"State Code": "State"})


median_df_new = pd.DataFrame({"Median household income":median_df.groupby(["State","County"])["Median household income"].mean(),
                              "Population":median_df.groupby(["State","County"])["Population"].sum()})

median_df_new.head()

Median household income  Population
State County                                                         
AK    Aleutians East Borough                      61518.0        3304
      Aleutians West Census Area                  84306.0        5684
      Anchorage Municipality                      78326.0      299107
      Bethel Census Area                          51012.0       17776
      Bristol Bay Borough                         79750.0         970

# Merge Datasets

In [63]:
# unemp_df_new - Final unemployment table with groupby on State
# median_df_new - Final Income table with groupby on State 
# new_df2 - Final diversity table with groupby on State 

# merged1 = pd.merge(unemp_df_new,median_df_new, on=["State",'County'] ,how="inner")

In [64]:
# merged1.head()

Unemployment Rate  Median household income  Population
State County                                                                
AL    Autauga County           5.417284                  51281.0       55221
      Baldwin County           5.364198                  50254.0      195121
      Barbour County           8.460185                  32964.0       26932
      Bibb County              7.684877                  38678.0       22604
      Blount County            5.164506                  45813.0       57710

In [75]:
# merged2 = pd.merge(merged1,new_df2, on=["State",'County'],how="outer")

In [76]:
# merged2.tail()

Unemployment Rate  Median household income  \
State County                                                          
 WY   Sweetwater County                NaN                      NaN   
      Teton County                     NaN                      NaN   
      Uinta County                     NaN                      NaN   
      Washakie County                  NaN                      NaN   
      Weston County                    NaN                      NaN   

                         Population  Diversity-Index  Black  American Indian  \
State County                                                                   
 WY   Sweetwater County         NaN         0.340296    1.7              1.2   
      Teton County              NaN         0.325230    1.5              1.0   
      Uinta County              NaN         0.225552    0.8              1.3   
      Washakie County           NaN         0.296868    0.8              1.7   
      Weston County             NaN         0.147697    0.6              1.7   

                         Asian  Native Hawaiian  Two or More Races  \
State County                                                         
 WY   Sweetwater County    0.9              0.2                1.7   
      Teton County         1.3              0.1                1.5   
      Uinta County         0.4              0.2                1.7   
      Washakie County      0.7              0.1                1.7   
      Weston County        0.3              0.0                2.1   

                         Hispanic/ Latino  White  
State County                                      
 WY   Sweetwater County              15.9   79.6  
      Teton County                   15.1   80.7  
      Uinta County                    9.1   87.5  
      Washakie County                14.2   82.6  
      Weston County                   3.8   92.2

In [77]:
# merged3 = merged2.reset_index()

In [81]:
# merged3.tail()

,State,County,Unemployment Rate,Median household income,Population,Diversity-Index,Black,American Indian,Asian,Native Hawaiian,Two or More Races,Hispanic/ Latino,White
5944,WY,Sweetwater County,NaN,NaN,NaN,0.340296,1.7,1.2,0.9,0.2,1.7,15.9,79.6
5945,WY,Teton County,NaN,NaN,NaN,0.325230,1.5,1.0,1.3,0.1,1.5,15.1,80.7
5946,WY,Uinta County,NaN,NaN,NaN,0.225552,0.8,1.3,0.4,0.2,1.7,9.1,87.5
5947,WY,Washakie County,NaN,NaN,NaN,0.296868,0.8,1.7,0.7,0.1,1.7,14.2,82.6
5948,WY,Weston County,NaN,NaN,NaN,0.147697,0.6,1.7,0.3,0.0,2.1,3.8,92.2


In [79]:
# merged4 = merged3.dropna()


In [80]:
# merged4.head()

,State,County,Unemployment Rate,Median household income,Population,Diversity-Index,Black,American Indian,Asian,Native Hawaiian,Two or More Races,Hispanic/ Latino,White
